# Функторы

Функция `map` была бы логична не только для списков. Как бы она работала для Maybe?

In [1]:
f x = x + 1

x1 = Just 42
x2 = Nothing

-- хочу применить f к x1, x2, точнее, к значению внутри
-- maybeMap f x1     ->    Just 43
-- maybeMap f x2     ->    Nothing

maybeMap fun (Just n) = Just (fun n)
maybeMap _ Nothing = Nothing

print $ maybeMap f x1
print $ maybeMap f x2

Just 43

Nothing

Допустим, я хочу уметь так же применять функции ко вторым элементам пары:

In [2]:
f x = x + 1
x1 = ("hello", 42)
x2 = ([1, 2, 3], 42.0)

-- хочу применить f ко второму элементу пары

pairMap fun (x, y) = (x, fun y)

print $ pairMap f x1
print $ pairMap f x2

("hello",43)

([1,2,3],43.0)

Есть и другие аналогичные ситуации. Когда значение (или значения) хранятся в какой-то структуре с дополнительными данными.
Список - несколько значений и информация об их порядке
Maybe - одно значение или ноль
Пара - значение вместе с дополнительной информацией

Функции тоже можно понимать как такие структуры:
fun :: a -> b
получается, функция хранит значения типа b, доступ к которым осуществляется по аргументу типа a:

Например,
```
f 1 = "один"
f 2 = "два"
...
```
функция "хранит" внутри себя значения `"один"`, `"два"`, получить их можно, подставив соответствующий аргумент в функцию.

In [3]:
f x = x + 1
x1 = \x -> 2 * x
x2 = \x -> 3 * x

-- funMap f x1 - должна получиться функция, которая совпадает с x1, но после применения x1 дополнительно вызывает f

funMap fun x = \t -> fun (x t)

xx1 = funMap f x1
xx2 = funMap f x2
print $ xx1 42
print $ xx2 42

85

127

На самом деле все эти map функции уже есть. Они называются fmap или <$>, и они делают ровно то же, что делаем мы:

In [4]:
print $ fmap (+1) (Just 42)
print $ (+1) <$> Just 42

print $ (+1) <$> [10, 20, 30]
print $ (+1) <$> ("abc", 123)

Just 43

Just 43

[11,21,31]

("abc",124)

Это работает, потому что в Haskell есть класс типов Functor - функтор. Он определен так:

In [5]:
class Functor' f where
  fmap' :: (a -> b) -> f a -> f b

У него есть реализации, например, примерно так выглядит реализация для Maybe:

In [0]:
instance Functor Maybe where
  fmap f (Just x) = Just (f x)
  fmap _ Nothing = Nothing

Если вы определяете как функтор, вы должны гарантировать выполнение следующего правила. Функция `id` - это функция, которая ничего не делает: `id x = x`, ее тип `a -> a`.

**Правила**:
1. `fmap id x == x` (или эквивалентно) `fmap id == id`
1. `fmap (g . f) == fmap g . fmap f`

## Аппликативные функторы

Расширение обычных функторов дополнительными операциями. Представьте опять же Maybe.
Было в функторах:
1. Обычная функция
2. Данные внутри структуры
3. Применяем функцию к каждому данному внутри структуры

Теперь в аппликативных функторах можно дополнительно
1. Обычные функции внутри структуры
2. Данные внутри структуры
3. Применяем функции из структуры к данным. Как именно применять - очень сильно зависит от ситуации.

Пример с Maybe:

In [8]:
f = Just (+1) -- увеличение на 1, но внутри Maybe
x = Just 42 -- значение внутри Maybe

-- применение:
maybeApply (Just fun) (Just x) = Just (fun x)
maybeApply _ _ = Nothing

print $ maybeApply f x
print $ maybeApply Nothing x
print $ maybeApply (Just (*2)) Nothing


Just 43

Nothing

Nothing

А для списков?

In [10]:
f = [(+1), (*2)]
x = [10, 20, 30]

-- что может означать listApply, когда есть список функций и список значений?
-- Применить каждую функцию к каждому элементу, получить список новых значений.

listApply fs xs = concatMap (\f -> map f xs) fs

print $ listApply f x

[11,21,31,20,40,60]

Немного философии, почему для списков это логично делать именно так. Можно считать, что Maybe - это вычисления, в которых иногда отсутствуют результаты. А в списках можно считать, что мы делаем вычисления, в которых может быть несколько результатов.

В Haskell есть класс типов Аппликативные функторы:

In [0]:
class Functor' f => Applicative' f where
    pure :: a -> f a -- вставить значение в функтор
    (<*>) :: f (a -> b) -> f a -> f b -- та самая функция применения, рассмотренная выше
    -- и еще несколько, см. дальше

`pure` - это функция, которая помещает значение внутрь функтора, "самым естественным способом". Что это значит будет понятно из требований к аппликативному функтору, см. дальше.

Для Maybe: `pure x = Just x`, для списка `pure x = [x]`

Допустим, хотим сложить значения в двух Maybe. Или в двух списках сложить попарно значения. Как это будет написано?

In [11]:
x1 = Just 110
x2 = Just 654

-- надо сложить. Сначала давайте получим Just (+110), потом применим ее к Just 654:

(+) 110